In [132]:
# list of netcdf files containing a Variable "MOD08_D3_6_1_Cloud_Fraction_Day_Mean"
# want to perform some analysis on these files
# and then output to a new netcdf file with the same dimensions and headers

# import libraries
import numpy as np
import netCDF4 as nc
import os
import glob
import matplotlib.pyplot as plt
import datetime as dt
import pandas as pd
# add type hints

# define function to get the data from the netcdf file
def get_data(filen: str):
    """
    Get the data from the netcdf file
    """
    # open the netcdf file
    with nc.Dataset(filen) as data:
    # get the data from the netcdf file
        try:
            var = data.variables['MOD08_D3_6_1_Cloud_Fraction_Day_Mean'][:]
        except KeyError:
            print(filen)
            return None
        # return the data
        return var



def analysis(data: np.ndarray):
    """
    Perform some analysis on the data
    """
    # get the mean of the data
    # mean = np.mean(data, axis=0)
    mean = np.mean(data, axis=0)
    mean = np.ma.filled(mean, fill_value=-9999.0)
    mean = np.ma.masked_equal(mean, -9999.0)
    # return the mean
    return mean

def cloud_probability(data: np.ndarray, cloud_frac: float):
    """
    Calculate the cloud probability
    P(f > cloud_frac)
    """
    # get the number of observations
    n_obs = data.shape[0]
    # get the number of observations where the cloud fraction is greater than the threshold
    n_cloud = np.sum(data > cloud_frac, axis=0)
    # calculate the cloud probability
    cloud_prob = n_cloud / n_obs
    # return the cloud probability
    return cloud_prob


In [133]:
all_data = []
# get the data from all the files
filenames = glob.glob('data/scrub*.nc')
for filen in filenames:
    all_data.append(get_data(filen))
    
# convert the list to an array  
all_data_m = np.ma.masked_array(all_data)

In [134]:
median = np.ma.median(all_data_m, axis=0)[0]
mean = np.ma.mean(all_data_m, axis=0)[0]

percentile = lambda x: np.ma.masked_invalid(np.nanpercentile(np.ma.filled(all_data_m, np.nan), x, axis=0))
quantile_75 = percentile(75)
quantile_25 = percentile(25)

/Users/jal194/anaconda3/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1577: RuntimeWarning: All-NaN slice encountered
  result = np.apply_along_axis(_nanquantile_1d, axis, a, q,


In [85]:
# function to slice the data and create a new variable in the netcdf file
def slice_data(dataset, var,latbounds, lonbounds, return_lat_lon=False):
    """
    Slice the data and create a new variable in the netcdf file
    """
    
    lats = dataset.variables['lat'][:] 
    lons = dataset.variables['lon'][:]
    
    latbounds = list(sorted(latbounds))
    lonbounds = list(sorted(lonbounds))

    # latitude lower and upper index
    latli = np.argmin( np.abs( lats - latbounds[0] ) )
    latui = np.argmin( np.abs( lats - latbounds[1] ) ) 

    # longitude lower and upper index
    lonli = np.argmin( np.abs( lons - lonbounds[0] ) )
    lonui = np.argmin( np.abs( lons - lonbounds[1] ) )  
    # slice the data
    
    new_lat = dataset.variables['lat'][latli:latui]
    new_lon = dataset.variables['lon'][lonli:lonui]
    
    if return_lat_lon:
        return dataset.variables[var][ latli:latui , lonli:lonui ], new_lat, new_lon
    else:
        return dataset.variables[var][ latli:latui , lonli:lonui ] 

In [136]:
def create_basic_netCDFfile(filen, dim_source = None, lat = None, lon = None, data = None, name = 'name', format = 'f4', units = '', long_name = 'Value'):
    # if the file exists, delete it
    if os.path.exists(filen):
        os.remove(filen)

    # create the netcdf file
    dataset = nc.Dataset(filen, 'w', format='NETCDF4')
    # create the dimensions
    if dim_source is None:
        dataset.createDimension('lat', len(lat))
        dataset.createDimension('lon', len(lon))
        
        if lat is not None:
            latitude = dataset.createVariable('lat', 'f4', ('lat',))
            # add the attributes
            latitude.units = 'degrees_north'
            latitude.long_name = 'latitude'
            # add the data to the variables
            latitude[:] = lat
            
        
        if lon is not None:
            longitude = dataset.createVariable('lon', 'f4', ('lon',))
            longitude.units = 'degrees_east'
            longitude.long_name = 'longitude'
            longitude[:] = lon
    else:
         # copy the dimensions and variables from the source file
        for dname, the_dim in dim_source.dimensions.items():
            dataset.createDimension(dname, len(the_dim) if not the_dim.isunlimited() else None)
        for v_name, varin in dim_source.variables.items():
            # only create the variables that are dimensions in the source file
            if v_name in dim_source.dimensions:
                outVar = dataset.createVariable(v_name, varin.datatype, varin.dimensions)
                # Copy variable attributes
                outVar.setncatts({k: varin.getncattr(k) for k in varin.ncattrs()})
                outVar[:] = varin[:]
    
    
    # create a geogreferenced variable
    if data is not None:
        newVar = dataset.createVariable(name, format, ('lat', 'lon'))
        # add the attributes
        newVar.units = units
        newVar.long_name = long_name
        # add the lat and lon variables
        
        newVar[:] = data


    return dataset

def add_variable_to_dataset(dataset, variable_data, name = 'value', units = '', fmt = 'f4', dimensions = ('lat', 'lon'), long_name = 'Value'):
    """
    Add a variable to a dataset
    """
    # create a geogreferenced variable
    newVar = dataset.createVariable(name, fmt, dimensions)
    # add the attributes
    newVar.units = units
    newVar.long_name = long_name
    # add the lat and lon variables
    newVar[:] = variable_data
    return dataset

In [137]:
# create a new netcdf file
# create the netcdf file
with nc.Dataset(filenames[0], 'r', format='NETCDF4') as dim_source:
    new_netcdf = create_basic_netCDFfile('new_netcdf.nc', dim_source=dim_source)

# create the variables
vars = {
    'mean': (mean, 'Mean Cloud Fraction'), 
    'median': (median, 'Median Cloud Fraction'),
    'quantile_75': (quantile_75, '75th Percentile Cloud Fraction'),
    'quantile_25': (quantile_25, '25th Percentile Cloud Fraction')
}

for var in vars:
    add_variable_to_dataset(new_netcdf, vars[var][0], name=var, units='percentage', dimensions=('lat', 'lon'), long_name=vars[var][1])

# new_netcdf.close()


In [138]:
latbounds = [14, 52]
lonbounds = [-130, -60]

# Slice the data and create variables for each variable in the vars dictionary
sliced_data = {}
for var in vars:
    sliced_data[var] = slice_data(new_netcdf, var, latbounds=latbounds, lonbounds=lonbounds, return_lat_lon=False)

_, sliced_lat, sliced_lon = slice_data(new_netcdf, 'mean', latbounds=latbounds, lonbounds=lonbounds, return_lat_lon=True)
# Create a new netcdf file and add the sliced variables to it
sliced_dataset = create_basic_netCDFfile('new_netcdf2.nc', 
                                  lat = sliced_lat,
                                  lon = sliced_lon,)

for var in vars:
    print(var)
    add_variable_to_dataset(sliced_dataset, sliced_data[var], name=var, units='percentage', fmt='f4', dimensions=('lat', 'lon'))


sliced_dataset.close()

mean
median
quantile_75
quantile_25


In [155]:
out_arr = np.zeros((len(sliced_lat)+1, len(sliced_lon)+1))
out_arr[0,0] = 9999
out_arr[1:,1:] = sliced_data['mean']
out_arr[0,1:] = sliced_lon
out_arr[1:,0] = sliced_lat

# write out out_arr to a csv file, with trailing zeros trimmed
np.savetxt('out.csv', out_arr, delimiter=',', fmt='%0.2f')
